----
<img src="../../../files/refinitiv.png" width="20%" style="vertical-align: top;">

# Data Library for Python

----

## Content layer - How to send parallel requests and receive results as events
This notebook demonstrates how to use Definition objects of the Content layer to send parallel requests and receive results as separated events. Historical Pricing will be used here as an example, but this technique applies to all Definition objects of the Content layer.   

#### Learn more

To learn more about the Refinitiv Data Library for Python please join the Refinitiv Developer Community. By [registering](https://developers.refinitiv.com/iam/register) and [logging](https://developers.refinitiv.com/content/devportal/en_us/initCookie.html) into the Refinitiv Developer Community portal you will have free access to a number of learning materials like 
 [Quick Start guides](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/quick-start), 
 [Tutorials](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/learning), 
 [Documentation](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/docs)
 and much more.

#### Getting Help and Support

If you have any questions regarding using the API, please post them on 
the [Refinitiv Data Q&A Forum](https://community.developers.refinitiv.com/spaces/321/index.html). 
The Refinitiv Developer Community will be happy to help. 

## Set the configuration file location
For a better ease of use, you have the option to set initialization parameters of the Refinitiv Data Library in the _refinitiv-data.config.json_ configuration file. This file must be located beside your notebook, in your user folder or in a folder defined by the _RD_LIB_CONFIG_PATH_ environment variable. The _RD_LIB_CONFIG_PATH_ environment variable is the option used by this series of examples. The following code sets this environment variable.      

In [55]:
import os
os.environ["RD_LIB_CONFIG_PATH"] = "../../../Configuration"

## Some Imports to start with

In [56]:
import refinitiv.data as rd
from refinitiv.data.content import historical_pricing
import asyncio
import datetime

## Open the data session

The open_session() function creates and open sessions based on the information contained in the refinitiv-data.config.json configuration file. Please edit this file to set the session type and other parameters required for the session you want to open.

In [57]:
rd.open_session()

<refinitiv.data.session.Definition object at 0x7f971a8b3390 {name='workspace'}>

## Retrieve data

### Define a callback to receive events and display responses

In [58]:
def display_response(response, definition, session):
    current_time = datetime.datetime.now().time()
    display('{} response received at {}'.format(response.closure, current_time))
    if response.is_success:
        display(response.data.df)
    else:
        print(response.http_status)    

### Use get_data_async to send requests in parallel and use asyncio to create separated tasks 
Once done, just wait for reponses to be received and displayed...

In [59]:
asyncio.get_event_loop().create_task(
    historical_pricing.events.Definition(
        universe = 'LSEG.L',
        closure = 'LSEG.L Historical Events'
    ).get_data_async(on_response = display_response)
)

asyncio.get_event_loop().create_task(
    historical_pricing.summaries.Definition(
        universe = 'LSEG.L',
        closure = 'LSEG.L Historical Summaries'
    ).get_data_async(on_response = display_response)
)

asyncio.get_event_loop().create_task(
    historical_pricing.summaries.Definition(
        universe = 'LSEG.L',
        closure = 'LSEG.L Historical Summaries'
    ).get_data_async(on_response = display_response)
)

<Task pending coro=<DataProviderLayer.get_data_async() running at /opt/anaconda3/lib/python3.7/site-packages/refinitiv/data/_data/tools/_raise_exception.py:34>>

'LSEG.L Historical Summaries response received at 14:04:00.459873'

LSEG.L,TRDPRC_1,MKT_HIGH,MKT_LOW,ACVOL_UNS,MKT_OPEN,BID,ASK,TRNOVR_UNS,VWAP,MID_PRICE,...,CLS_AUCVOL,OPN_AUCVOL,OPN_AUC,CLS_AUC,INT_AUC,INT_AUCVOL,EX_VOL_UNS,ALL_C_MOVE,ELG_NUMMOV,NAVALUE
Date,,,,,,,,,,,,,,,,,,,,,
2021-11-18,6728.7024,6839.7811,6680,697709,6700,6680,6684,4694048229.6,6727.80232,6682,...,235971,18712,6700,6680,<NA>,<NA>,725223,6648,5982,None
2021-11-19,6680.0,6790.0,6636,1227919,6680,6702,6704,8217046746.09,6691.84744,6703,...,418259,13540,6680,6704,6726,20380,1282403,6832,6323,None
2021-11-22,6684.0,6856.0,6684,625896,6736,6684,6686,4217182628.2061,6737.83285,6685,...,242263,13647,6736,6684,<NA>,<NA>,688369,6840,6363,None
2021-11-23,6734.0,6766.0,6600,642256,6668,6734,6736,4303886557.0328,6701.20101,6735,...,242516,3217,6668,6734,<NA>,<NA>,694941,7210,6752,None
2021-11-24,6824.0,6832.0,6552,1809481,6780,6810,6812,12229515457.057541,6758.57626,6811,...,276206,9045,6780,6810,<NA>,<NA>,1943949,10446,9849,None
2021-11-25,6822.0,6872.0,6738,982614,6822,6824,6826,6702808788.36,6821.40575,6825,...,198533,2155,6822,6824,<NA>,<NA>,1141701,5848,5560,None
2021-11-26,6704.0,6880.52,6680,946540,6704,6824,6850,6433414329.78602,6796.76963,6837,...,230979,5822,6704,6850,<NA>,<NA>,1084983,9173,8736,None
2021-11-29,6764.3,6926.0,6754,1119681,6890,6870,6872,7678850720.20644,6858.06992,6871,...,340426,4129,6890,6870,<NA>,<NA>,1293553,8653,8291,None
2021-11-30,6502.0,6872.0,6502,2023297,6870,6502,6512,13234527793.25,6541.07024,6507,...,1560106,9295,6870,6502,<NA>,<NA>,2495199,7992,7365,None


'LSEG.L Historical Summaries response received at 14:04:00.848837'

LSEG.L,TRDPRC_1,MKT_HIGH,MKT_LOW,ACVOL_UNS,MKT_OPEN,BID,ASK,TRNOVR_UNS,VWAP,MID_PRICE,...,CLS_AUCVOL,OPN_AUCVOL,OPN_AUC,CLS_AUC,INT_AUC,INT_AUCVOL,EX_VOL_UNS,ALL_C_MOVE,ELG_NUMMOV,NAVALUE
Date,,,,,,,,,,,,,,,,,,,,,
2021-11-18,6728.7024,6839.7811,6680,697709,6700,6680,6684,4694048229.6,6727.80232,6682,...,235971,18712,6700,6680,<NA>,<NA>,725223,6648,5982,None
2021-11-19,6680.0,6790.0,6636,1227919,6680,6702,6704,8217046746.09,6691.84744,6703,...,418259,13540,6680,6704,6726,20380,1282403,6832,6323,None
2021-11-22,6684.0,6856.0,6684,625896,6736,6684,6686,4217182628.2061,6737.83285,6685,...,242263,13647,6736,6684,<NA>,<NA>,688369,6840,6363,None
2021-11-23,6734.0,6766.0,6600,642256,6668,6734,6736,4303886557.0328,6701.20101,6735,...,242516,3217,6668,6734,<NA>,<NA>,694941,7210,6752,None
2021-11-24,6824.0,6832.0,6552,1809481,6780,6810,6812,12229515457.057541,6758.57626,6811,...,276206,9045,6780,6810,<NA>,<NA>,1943949,10446,9849,None
2021-11-25,6822.0,6872.0,6738,982614,6822,6824,6826,6702808788.36,6821.40575,6825,...,198533,2155,6822,6824,<NA>,<NA>,1141701,5848,5560,None
2021-11-26,6704.0,6880.52,6680,946540,6704,6824,6850,6433414329.78602,6796.76963,6837,...,230979,5822,6704,6850,<NA>,<NA>,1084983,9173,8736,None
2021-11-29,6764.3,6926.0,6754,1119681,6890,6870,6872,7678850720.20644,6858.06992,6871,...,340426,4129,6890,6870,<NA>,<NA>,1293553,8653,8291,None
2021-11-30,6502.0,6872.0,6502,2023297,6870,6502,6512,13234527793.25,6541.07024,6507,...,1560106,9295,6870,6502,<NA>,<NA>,2495199,7992,7365,None


'LSEG.L Historical Events response received at 14:04:01.139842'

LSEG.L,EVENT_TYPE,RTL,SEQNUM,TRDXID_1,TRDPRC_1,TRDVOL_1,VWAP,BID,BIDSIZE,ASK,...,MMT_CLASS,TR_TRD_FLG,ACVOL_UNS,OPEN_PRC,HIGH_1,LOW_1,MID_PRICE,IMB_SH,IMB_SIDE,QUALIFIERS
Timestamp,,,,,,,,,,,,,,,,,,,,,
2021-12-16 13:03:18.316,quote,33408,2862845,None,None,None,None,6828,83,6832,...,None,None,None,None,None,None,6830,None,None,[ASK_TONE]
2021-12-16 13:03:18.316,quote,33424,2862846,None,None,None,None,6828,83,6832,...,None,None,None,None,None,None,6830,None,None,[ASK_TONE]
2021-12-16 13:03:18.317,quote,33440,2862859,None,None,None,None,6828,83,6832,...,None,None,None,None,None,None,6830,None,None,[ASK_TONE]
2021-12-16 13:03:18.317,quote,33456,2862860,None,None,None,None,6828,126,6832,...,None,None,None,None,None,None,6830,None,None,[BID_TONE]
2021-12-16 13:03:19.532,quote,33472,2863055,None,None,None,None,6828,126,6832,...,None,None,None,None,None,None,6830,None,None,[ASK_TONE]
2021-12-16 13:03:19.613,quote,33488,2863057,None,None,None,None,6828,126,6830,...,None,None,None,None,None,None,6829,None,None,[ASK_TONE]
2021-12-16 13:03:19.903,quote,33504,2863103,None,None,None,None,6828,126,6830,...,None,None,None,None,None,None,6829,None,None,[ASK_TONE]
2021-12-16 13:03:19.903,quote,33520,2863104,None,None,None,None,6828,76,6830,...,None,None,None,None,None,None,6829,None,None,[BID_TONE]
2021-12-16 13:03:19.903,quote,33536,2863105,None,None,None,None,6828,43,6830,...,None,None,None,None,None,None,6829,None,None,[BID_TONE]


## Close the session

In [60]:
rd.close_session()